# Análisis exploratorio

## Carga de los datos

In [29]:
import pandas as pd
from pathlib import Path
import pickle

In [30]:
dir_actual = Path('.')
if not (dir_actual / 'dict_dataframes_short.pkl').exists():
    directorio = Path('../datos')
    dataframes_short = {}
    for f in directorio.iterdir():
        if f.suffix == '.xlsx':
            print(f)
            xlsx = pd.ExcelFile(f)
            for sheet in xlsx.sheet_names:
                if sheet != 'WData':
                    df = xlsx.parse(sheet)
                    dataframes_short[str(f).split('/')[-1]+"/"+sheet] = df
with open('dict_dataframes_short.pkl', 'wb') as file:
        pickle.dump(dataframes_short, file)

../datos/Centrífuga 17825.xlsx
../datos/Biorreactor 13169.xlsx
../datos/Fases producción v02.xlsx
../datos/Cinéticos IPC.xlsx
../datos/Horas inicio fin centrífugas.xlsx
../datos/Centrífuga 14246.xlsx
../datos/Fases producción_test v02.xlsx
../datos/Biorreactor 14617.xlsx
../datos/Biorreactor 13171.xlsx
../datos/Biorreactor 14614.xlsx
../datos/Biorreactor 13170.xlsx
../datos/Biorreactor 14615.xlsx
../datos/OF 123456 v02.xlsx
../datos/Biorreactor 14618.xlsx
../datos/Movimientos componentes.xlsx
../datos/Biorreactor 13172.xlsx
../datos/Temperaturas y humedades.xlsx
../datos/Biorreactor 14616.xlsx
../datos/Centrífuga 12912.xlsx


In [31]:
dir_actual = Path('.')
if (dir_actual / 'dict_dataframes.pkl').exists():
    dataframes = pd.read_pickle('dict_dataframes.pkl')
    print('Leyendo datos de dict_dataframes.pkl')
else:
    directorio = Path('../datos')
    dataframes = {}
    for f in directorio.iterdir():
        if f.suffix == '.xlsx':
            print(f)
            xlsx = pd.ExcelFile(f)
            for sheet in xlsx.sheet_names:
                if sheet != 'WData':
                    df = xlsx.parse(sheet)
                    dataframes[str(f)+"/"+sheet] = df

../datos/Centrífuga 17825.xlsx
../datos/Biorreactor 13169.xlsx
../datos/Fases producción v02.xlsx
../datos/Cinéticos IPC.xlsx
../datos/Horas inicio fin centrífugas.xlsx
../datos/Centrífuga 14246.xlsx
../datos/Fases producción_test v02.xlsx
../datos/Biorreactor 14617.xlsx
../datos/Biorreactor 13171.xlsx
../datos/Biorreactor 14614.xlsx
../datos/Biorreactor 13170.xlsx
../datos/Biorreactor 14615.xlsx
../datos/OF 123456 v02.xlsx
../datos/Biorreactor 14618.xlsx
../datos/Movimientos componentes.xlsx
../datos/Biorreactor 13172.xlsx
../datos/Temperaturas y humedades.xlsx
../datos/Biorreactor 14616.xlsx
../datos/Centrífuga 12912.xlsx


In [32]:
dataframes.keys()

dict_keys(['../datos/Centrífuga 17825.xlsx/Datos', '../datos/Biorreactor 13169.xlsx/Datos', '../datos/Fases producción v02.xlsx/Preinóculo', '../datos/Fases producción v02.xlsx/Inóculo', '../datos/Fases producción v02.xlsx/Cultivo final', '../datos/Cinéticos IPC.xlsx/Inóculos', '../datos/Cinéticos IPC.xlsx/Cultivos finales', '../datos/Cinéticos IPC.xlsx/Centrifugación', '../datos/Horas inicio fin centrífugas.xlsx/Hoja1', '../datos/Centrífuga 14246.xlsx/Datos', '../datos/Fases producción_test v02.xlsx/Cultivo final', '../datos/Biorreactor 14617.xlsx/Datos', '../datos/Biorreactor 13171.xlsx/Datos', '../datos/Biorreactor 14614.xlsx/Datos', '../datos/Biorreactor 13170.xlsx/Datos', '../datos/Biorreactor 14615.xlsx/Datos', '../datos/OF 123456 v02.xlsx/Sheet1', '../datos/Biorreactor 14618.xlsx/Datos', '../datos/Movimientos componentes.xlsx/Full1', '../datos/Biorreactor 13172.xlsx/Datos', '../datos/Temperaturas y humedades.xlsx/Datos', '../datos/Biorreactor 14616.xlsx/Datos', '../datos/Centríf

Tenemos un diccionario con todos los dataframes de los ficheros para poder acceder de manera eficiente.

Ahora los guardamos en un archivo pickle para luego cargar los datos más rápido en próximas ejecuciones.

In [33]:
if not ((dir_actual / 'dict_dataframes.pkl').exists()):
    with open('dict_dataframes.pkl', 'wb') as f:
        pickle.dump(dataframes, f)

## Creación dataframe de exploración

Vamos a ir creando los dataframes de las variables numéricas de cada uno de los ficheros para luego juntarlos en uno, facilitando su análisis posterior.

In [34]:
# para limpiar los strings de los lotes y utilizar un formato común
def limpiar_string_lote(string: str) -> int:
    if isinstance(string, str):
        if ',' in string:
            string = string.replace(',', '')
        if '/' in string:
            string = string.replace('/', '')
        if 'P' in string:
            string = string.replace('P', '') # hay un lote que tiene una P al principio
        return int(string)
    return string
        

In [44]:
dict_of = {
    'Lote' : list(map(limpiar_string_lote ,dataframes['../datos/OF 123456 v02.xlsx/Sheet1']['Lote'].values)),
    'Cantidad entregada (L)' : dataframes['../datos/OF 123456 v02.xlsx/Sheet1']['Cantidad entregada'].values,
}

df_of = pd.DataFrame(dict_of)
df_of

,Lote,Cantidad entregada (L)
0,23019,13.80
1,23020,13.60
2,23021,13.50
3,23022,13.80
4,23023,13.70
...,...,...
202,24108,13.63
203,24106,13.69
204,24107,13.45
205,24109,11.35


In [47]:
dataframes['../datos/Fases producción v02.xlsx/Preinóculo']

,LOTE,Fecha/hora inicio,Fecha/hora fin,pH línea 1,pH línea 2,pH línea 3,Turbidez línea 1,Turbidez línea 2,Turbidez línea 3,Línea usada línea 1,Línea usada línea 2,Línea usada línea 3
1,23023,2023-03-26 05:00:00,2023-03-27 07:21:00,5.496,5.504,5.52,28.32,27.92,32,1,1,0
2,23024,2023-03-26 05:00:00,2023-03-27 07:21:00,5.496,5.504,5.52,28.32,27.92,32,1,1,0
3,23025,2023-03-17 06:00:00,2023-03-28 07:42:00,5.48,5.52,5.52,26.56,27.52,26.4,1,1,0
4,23026,2023-03-17 06:00:00,2023-03-28 07:42:00,5.48,5.52,5.52,26.56,27.52,26.4,1,1,0
5,23027,2023-04-02 05:00:00,2023-04-03 13:30:00,5.384,5.44,5.4,33.84,30.56,32.48,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
161,24103,2024-06-27 00:30:00,2024-06-28 07:16:00,5.424,5.432,5.416,32.08,32.48,30.64,1,1,0
162,24104,2024-06-30 00:30:00,2024-07-01 07:01:00,5.408,5.424,5.392,27.44,26.48,26.64,1,0,1
163,24105,2024-06-30 00:30:00,2024-07-01 07:01:00,5.408,5.424,5.392,27.44,26.48,26.64,1,0,1
164,24108,2024-07-04 00:30:00,2024-07-05 07:04:00,5.528,5.432,5.504,33.44,28.8,25.6,0,1,1


Nos encontramos un problema en la hoja de Preinóculo de Fases de producción.xlsx con el formato de las cabeceras de las columnas, por eso lo corregimos.

In [45]:
new_columns = ['LOTE', 'Fecha/hora inicio', 'Fecha/hora fin', 'pH línea 1',
       'pH línea 2', 'pH línea 3', 'Turbidez línea 1', 'Turbidez línea 2', 'Turbidez línea 3', 'Línea usada línea 1',
       'Línea usada línea 2', 'Línea usada línea 3']

In [48]:
dataframes['../datos/Fases producción v02.xlsx/Preinóculo'].columns = new_columns

In [49]:
dict_fases_prod_preinoculo = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['LOTE'].values)),
    'ph 1' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['pH línea 1'].values,
    'ph 2' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['pH línea 2'].values,
    'ph 3' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['pH línea 3'].values,
    'turbidez 1' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Turbidez línea 1'].values,
    'turbidez 2' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Turbidez línea 2'].values,
    'turbidez 3' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Turbidez línea 3'].values,
    'linea 1' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Línea usada línea 1'].values,
    'linea 2' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Línea usada línea 2'].values,
    'linea 3' : dataframes['../datos/Fases producción v02.xlsx/Preinóculo']['Línea usada línea 3'].values,
}

dict_fases_prod_inoculo = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción v02.xlsx/Inóculo']['LOTE'].values)),
    'Volumen de cultivo' : dataframes['../datos/Fases producción v02.xlsx/Inóculo']['Volumen de cultivo'].values,
    'Turbidez inicial cultivo inóculo' : dataframes['../datos/Fases producción v02.xlsx/Inóculo']['Turbidez inicio cultivo'].values,
    'Turbidez final cultivo inóculo' : dataframes['../datos/Fases producción v02.xlsx/Inóculo']['Turbidez final culttivo'].values,
    'Viabilidad final cultivo' : dataframes['../datos/Fases producción v02.xlsx/Inóculo']['Viabilidad final cultivo'].values,
}

dict_fases_prod_cultivo_final = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['LOTE'].values)),
    'Orden en el encadenado' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Orden en el encadenado'].values,
    'Volumen de inóculo utilizado' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Volumen de inóculo utilizado'].values,
    'Turbidez inicio cultivo final' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Turbidez inicio cultivo'].values,
    'Turbidez final cultivo final' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Turbidez fin cultivo'].values,
    'Centrifugación 1 turbidez' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Centrifugación 1 turbidez'].values,
    'Centrigugación 2 turbidez' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Centrifugación 2 turbidez'].values,
    'Producto 1' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Producto 1'].values,
    'Producto 2' : dataframes['../datos/Fases producción v02.xlsx/Cultivo final']['Producto 2'].values,
}


df_fases_prod_preinoculo = pd.DataFrame(dict_fases_prod_preinoculo)
df_fases_prod_inoculo = pd.DataFrame(dict_fases_prod_inoculo)
df_fases_prod_cultivo_final = pd.DataFrame(dict_fases_prod_cultivo_final)

Ahora vamos a hacer un join de todos los dataframes para tener un único archivo donde vengan recogidas todas las medidas de un lote a lo largo de su producción. No tenemos en cuenta las mediciones en biorreactores ni cinéticas ya que se trata de una análisis exploratorio inicial.

In [50]:
df = pd.merge(df_of, df_fases_prod_preinoculo, on='Lote', how='inner')
df = pd.merge(df, df_fases_prod_inoculo, on='Lote', how='inner')
df = pd.merge(df, df_fases_prod_cultivo_final, on='Lote', how='inner')
df

,Lote,Cantidad entregada (L),ph 1,ph 2,ph 3,turbidez 1,turbidez 2,turbidez 3,linea 1,linea 2,...,Turbidez final cultivo inóculo,Viabilidad final cultivo,Orden en el encadenado,Volumen de inóculo utilizado,Turbidez inicio cultivo final,Turbidez final cultivo final,Centrifugación 1 turbidez,Centrigugación 2 turbidez,Producto 1,Producto 2
0,23023,13.70,5.496,5.504,5.52,28.32,27.92,32,1,1,...,31.68,106400000,1,77.60,17.12,74.40,26.56,20.88,1861.84,2.96
1,23024,13.70,5.496,5.504,5.52,28.32,27.92,32,1,1,...,31.68,106400000,1,76.00,16.56,80.80,24.56,10.40,2161.12,2.80
2,23025,13.70,5.48,5.52,5.52,26.56,27.52,26.4,1,1,...,27.60,84800000,1,77.20,17.76,87.20,30.64,29.36,2044.72,4.48
3,23026,13.80,5.48,5.52,5.52,26.56,27.52,26.4,1,1,...,27.60,84800000,1,78.80,18.24,81.20,26.48,11.60,2263.20,3.44
4,23027,13.70,5.384,5.44,5.4,33.84,30.56,32.48,1,0,...,23.52,104800000,1,83.20,16.88,68.08,26.24,9.84,1407.68,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,24045,13.71,5.512,5.624,5.552,26.08,25.2,28.24,1,1,...,23.20,109600000,1,80.00,17.52,72.48,27.84,17.76,1573.52,5.76
119,24044,13.75,5.424,5.408,5.48,24.4,27.92,24.48,1,1,...,25.28,104800000,1,83.60,19.28,77.52,30.72,20.68,1528.72,5.44
120,24049,13.57,5.424,5.408,5.48,24.4,27.92,24.48,1,1,...,25.28,104800000,1,83.60,18.88,72.64,30.56,17.00,1342.80,4.88
121,24050,13.63,5.496,5.488,5.376,34,27.84,35.12,1,0,...,25.92,102400000,1,84.16,17.76,67.60,29.44,26.64,1422.80,3.68


In [51]:
df.to_csv('dataframe_exploratorio.csv', index=False)

## Buscar datos según fecha

In [52]:
from datetime import datetime

Dada una fecha, encuentra los datos de los biorreactores, centrifugadoras y de temperaturas y humedades rápidamentes.

In [53]:
def encontrar_subdataset_entre_fechas(df: pd.DataFrame, fecha_inicio: str, fecha_fin: str) -> pd.DataFrame:
    result = {}
    result['DateTime'] = df['DateTime'].map(lambda x: datetime.strptime(x[:-4], '%Y-%m-%d %H:%M:%S'))
    col_bool = (result['DateTime'] >= datetime.strptime(fecha_inicio, '%Y-%m-%d %H:%M:%S')) & (result['DateTime'] <= datetime.strptime(fecha_fin, '%Y-%m-%d %H:%M:%S'))
    return df[col_bool]

encontrar_subdataset_entre_fechas(dataframes['../datos/Biorreactor 13169.xlsx/Datos'], '2023-03-17 23:00:00', '2023-03-18 02:00:00')

,DateTime,13169_FERM0101.Agitation_PV,13169_FERM0101.Air_Sparge_PV,13169_FERM0101.Biocontainer_Pressure_PV,13169_FERM0101.DO_1_PV,13169_FERM0101.DO_2_PV,13169_FERM0101.Gas_Overlay_PV,13169_FERM0101.Load_Cell_Net_PV,13169_FERM0101.pH_1_PV,13169_FERM0101.pH_2_PV,13169_FERM0101.PUMP_1_PV,13169_FERM0101.PUMP_1_TOTAL,13169_FERM0101.PUMP_2_PV,13169_FERM0101.PUMP_2_TOTAL,13169_FERM0101.Single_Use_DO_PV,13169_FERM0101.Single_Use_pH_PV,13169_FERM0101.Temperatura_PV
284,2023-03-17 23:00:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.599628,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,16.146649
285,2023-03-17 23:15:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.496784,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,16.788062
286,2023-03-17 23:30:00.000,0.0,0.0,480.0,0.0,0.0,0.0,-19.504235,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,17.027268
287,2023-03-17 23:45:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.270780,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,17.838418
288,2023-03-18 00:00:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,18.057414
289,2023-03-18 00:15:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,18.079870
290,2023-03-18 00:30:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,8.894705,0.0,1486.330176,655.892432,799.623975,18.098775
291,2023-03-18 00:45:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,27.280002,0.0,1486.330176,655.892432,799.623975,18.094757
292,2023-03-18 01:00:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,27.280002,0.0,249.033884,655.892432,799.623975,18.012873
293,2023-03-18 01:15:00.000,0.0,0.0,480.0,0.0,NaN,0.0,-19.200000,1.56356,-0.011332,0.0,1.429191,0.0,1486.330176,655.892432,799.623975,18.075460


## Definición de las dimensiones de entrada en el modelo

Dadas las características del proceso, el modelo debe tener varias entradas para cada uno de los tipos de datos, como por ejemplo un input para las mediciones durante las fases de producción y otro input para los datos de registrados en el tiempo que ha estado en el biorreactor.